In [14]:
#import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [22]:
#read data
df=pd.read_csv('movies.csv')

In [23]:
# Remove duplicates in the entire DataFrame
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['title', 'release_date'], inplace=True)
df1 = df[df.vote_count >= 20].reset_index(drop=True)

#combining necessary columns
df1['tags'] = df1['overview'] +' '+ df1['genres'] +' '+ df1['keywords'] +' '+ df1['credits'] +' '+ df1['original_language']

# Drop rows with missing values
df1.dropna(inplace=True)

# Sampling
df1 = df1.sample(n=500, random_state=42)

In [24]:
# finding empty strings and removing them
index = df1[(df1['genres']=='') & (df1['overview']=='')].index
df1.drop(index, inplace=True)

# Removing punctuation
df1['tags'] = df1['tags'].str.replace('[^\w\s]', '', regex=True)

In [25]:
#defining tf-idf matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df1['tags'])

In [26]:
#Function to get user input
def get_user_input():
    while True:
        prompt = input("Describe the type of movie you are looking to watch: ")
        get_recommendations_from_prompt(prompt)
        continue_search = input("Do you want to search again? (yes/no): ").strip().lower()
        if continue_search != 'yes':
            print("Goodbye!")
            break

In [27]:
def get_recommendations_from_prompt(prompt, top_n=5):

    # Getting simmilar movies using cosine simmilarity
    prompt_vector = tfidf.transform([prompt])
    sim_scores = cosine_similarity(prompt_vector, tfidf_matrix).flatten()
    top_indices = sim_scores.argsort()[::-1][:top_n]


    recommended_movies = df1.iloc[top_indices]

    print("Top Recommended Movies:\n")
    for title in recommended_movies['title']:
        print(title)

get_user_input();


Describe the type of movie you are looking to watch: I want to watch a disney movie
Top Recommended Movies:

Rings
Summer Magic
Daniel, the Wizard
Treasure Island
The Pixar Story
Do you want to search again? (yes/no): yes
Describe the type of movie you are looking to watch: I want to watch a funny comedy film
Top Recommended Movies:

El Chanfle 2
Scooby Goes Hollywood
Vanity Fair
Cube Zero
Clerks
Do you want to search again? (yes/no): yes
Describe the type of movie you are looking to watch: I want to watch a gore horror thriller
Top Recommended Movies:

Hansel & Gretel
Centigrade
Infidel
Barricade
Hell Fest
Do you want to search again? (yes/no): no
Goodbye!
